In [57]:
import pandas as pd
#from modules.lavenshtein import lavenshtein, get_laven
from Levenshtein import distance
import os
from multiprocessing import Pool
from tqdm import tqdm
from modules.rev_helper import rev_helper

In [58]:
# test_df = pd.read_csv("data/original/test_transactions.csv")
# display(test_df)
# print(test_df.index.values.tolist())

In [59]:
'''
iterates through all the ocr files an finds the Levenshtein distance between the file's 
text content and the provided user data.
outputs a dataframe with the documentid, a list of potential userids, and a list of their
corresponding Levenshtein distances
'''
columns = ["BB1", "BB2", "BB3", "BB4", "BB5", "BB6", "BB7", "BB8", "Text_Main"]
columns_labels = ["documentid","pot_doc","pot_name","pot_price","pot_date","pot_address",]
ocrdir = 'data/interim/ocr/'
levan_user = pd.DataFrame(columns=columns_labels)
#reads user data
users_data = pd.read_csv("data/interim/Users.csv")
#removes all files that are not present in the user file
file_names = [os.path.splitext(file)[0] for file in list(os.listdir(ocrdir))]
file_names = [x for x in file_names if x in users_data['documentid'].values]
file_names = [file_name + '.csv' for file_name in file_names]
#pools the function so that it runs in paralell
with Pool() as p:
    q = p.map(rev_helper, zip(file_names, [users_data for _ in range(len(os.listdir(ocrdir)))]))
for qq in q:
    levan_user.loc[len(levan_user)] = qq

In [60]:
df = levan_user

def remove_element_at_position(value):
    if not value['position']:
        return pd.Series({'documentid':value['documentid'],'pot_doc':value['pot_doc'],'pot_name':value['pot_name'], 'pot_price':value['pot_price'],'pot_address':value['pot_address'],'pot_date':value['pot_date']})
    pot_doc = [value['pot_doc'][i] for i in value['position']]
    pot_name = [value['pot_name'][i] for i in value['position']]
    pot_price = [value['pot_price'][i] for i in value['position']]
    pot_address = [value['pot_address'][i] for i in value['position']]
    pot_date = [value['pot_date'][i] for i in value['position']]
    return pd.Series({'documentid':value['documentid'],'pot_doc':pot_doc,'pot_name':pot_name, 'pot_price':pot_price,'pot_address':pot_address,'pot_date':pot_date})
        

df['position'] = df['pot_date'].apply(lambda x: [i for i, x in enumerate(x) if x == 1])

df = df.apply(remove_element_at_position, axis=1)

display((df))
df.to_csv(os.path.join('data/final/', "final.csv"))


,documentid,pot_doc,pot_name,pot_price,pot_address,pot_date
0,00d0243046961,[00d0243046961],[9],[1],[93],[1]
1,00d0164592408,[00d0164592408],[2],[0],[92],[1]
2,00d0696518680,[00d0696518680],[1],[1],[97],[1]
3,00d0530724847,"[00d0191077139, 00d0530724847]","[0, 2]","[0, 1]","[92, 96]","[1, 1]"
4,00d0770154989,[00d0770154989],[14],[0],[93],[1]
...,...,...,...,...,...,...
494,00d0844880238,"[00d0841246769, 00d0333208815, 00d0821795963, ...","[3, 3, 3, 3, 3, 13, 16, 0, 3, 0, 0, 3, 3, 3, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[90, 93, 93, 90, 93, 93, 93, 93, 93, 93, 93, 9...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
495,00d0354051709,[00d0354051709],[12],[1],[95],[1]
496,00d0726523245,[00d0726523245],[8],[0],[93],[1]
497,00d0318453525,[00d0318453525],[14],[0],[100],[1]
